In [2]:
import math
import random

import numpy as np
import scipy.stats as stats
import snoop
from loguru import logger
from statsmodels.stats.proportion import proportion_confint

In [2]:
import bamboolib as bam

In [3]:
alpha = 0.003
snd = stats.norm()
z = snd.ppf(1 - 0.003 / 2)

print(round(189 / 11034 - (104 / 11037), 4))

print(np.round(z, 4))

asp, no_asp = np.array([1] * 104 + [0] * (11037 - 104)), np.array(
    [1] * 189 + [0] * (11034 - 189)
)

0.0077
2.9677


In [5]:
def get_bootstrap_samples(data, n_samples):
    random.seed(0)
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [6]:
with logger.catch():
    asp_samples, no_asp_samples = get_bootstrap_samples(
        asp, 1000
    ), get_bootstrap_samples(no_asp, 1000)
    asp_means, no_asp_means = list(map(np.mean, asp_samples)), list(
        map(np.mean, no_asp_samples)
    )
    delta_means = list(map(lambda x: x[0] - x[1], zip(asp_means, no_asp_means)))

In [7]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2.0, 100 * (1 - alpha / 2.0)])
    return boundaries

In [8]:
round(stat_intervals(delta_means, 0.05)[0], 4), round(
    stat_intervals(delta_means, 0.05)[1], 4
)

(-0.0106, -0.0048)

In [10]:
n1 = 11037
k1 = 104
p1 = float(k1) / n1
n2 = 11034
k2 = 189
p2 = float(k2) / n2
round(p2 - p1 + 1.96 * np.sqrt(p2 * (1 - p2) / n2 + p1 * (1 - p1) / n1), 4)

0.0107

In [11]:
odds_asp, odds_no_asp = 104 / 11037, 189 / 11034
odds_asp, odds_no_asp = odds_asp / (1 - odds_asp), odds_no_asp / (1 - odds_no_asp)

In [12]:
aspirin_n = 11037
placebo_n = 11034
aspirin_infarct_n = 104
placebo_infarct_n = 189

aspirin_p = float(aspirin_infarct_n) / aspirin_n
placebo_p = float(placebo_infarct_n) / placebo_n

In [13]:
def proportions_confint_diff_ind(p1, count1, p2, count2, alpha=0.05):
    z = stats.norm.ppf(1 - alpha / 2.0)

    left_boundary = (p1 - p2) - z * np.sqrt(
        p1 * (1.0 - p1) / count1 + p2 * (1 - p2) / count2
    )
    right_boundary = (p1 - p2) + z * np.sqrt(
        p1 * (1.0 - p1) / count1 + p2 * (1 - p2) / count2
    )

    return (left_boundary, right_boundary)

In [14]:
round((proportions_confint_diff_ind(aspirin_p, aspirin_n, placebo_p, placebo_n)[1]), 4)

-0.0047

In [15]:
def calculate_odds(success_count, full_count):
    return float(success_count) / float(full_count - success_count)


aspirin_odds = calculate_odds(aspirin_infarct_n, aspirin_n)
placebo_odds = calculate_odds(placebo_infarct_n, placebo_n)
print("Placebo to aspirin odds ratio: %.4f" % np.round(placebo_odds / aspirin_odds, 4))

Placebo to aspirin odds ratio: 1.8321


In [16]:
aspirin_vector = np.append(
    np.ones(int(aspirin_infarct_n), dtype=int),
    np.zeros(int(aspirin_n - aspirin_infarct_n), dtype=int),
)
placebo_vector = np.append(
    np.ones(int(placebo_infarct_n), dtype=int),
    np.zeros(int(placebo_n - placebo_infarct_n), dtype=int),
)

In [17]:
def get_bootstrap_samples(data, n_samples):
    data_length = len(data)
    indices = np.random.randint(0, data_length, (n_samples, data_length))
    return data[indices]


# %%
np.random.seed(0)
placebo_samples = get_bootstrap_samples(placebo_vector, 1000)
aspirin_samples = get_bootstrap_samples(aspirin_vector, 1000)

In [18]:
def calculate_odds_from_samples(samples):
    samples_count = len(samples)
    success_count = len(samples[samples == 1])
    return calculate_odds(success_count, samples_count)


# %%
all_aspirin_odds = list(map(calculate_odds_from_samples, aspirin_samples))
all_placebo_odds = list(map(calculate_odds_from_samples, placebo_samples))

# %%


def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2.0, 100 * (1 - alpha / 2.0)])
    return boundaries


# %%
odds = list(map(lambda x: x[1] / x[0], zip(all_aspirin_odds, all_placebo_odds)))
# %%
print(np.round(stat_intervals(odds, 0.05), 4))

[1.4629 2.3509]


In [24]:
import numpy as np
from scipy.stats import norm

# Давайте уточним правило трёх сигм.
# Утверждение: 99.7% вероятностной массы случайной величины X∼N(μ,σ^2) лежит в интервале μ±c⋅σ.
# Чему равно точное значение константы c?
# Округлите ответ до четырёх знаков после десятичной точки.
#%%
print("3 sigma rule refined: %.4f" % np.round(norm.ppf(0.003 / 2), 4))

# В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день
# принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали,
# что именно они принимают.
# За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.
# Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков
# после десятичной точки.
#%%
aspirin_n = 11037
placebo_n = 11034
aspirin_infarct_n = 104
placebo_infarct_n = 189

aspirin_p = float(aspirin_infarct_n) / aspirin_n
placebo_p = float(placebo_infarct_n) / placebo_n

print("Aspirin infarct probability: %.4f" % aspirin_p)
print("Placebo infarct probability: %.4f" % placebo_p)
print("Probability diff %.4f" % np.round(placebo_p - aspirin_p, 4))

#%%
from statsmodels.stats.proportion import proportion_confint

conf_interval_aspirin = proportion_confint(
    aspirin_infarct_n, aspirin_n, method="wilson"
)
conf_interval_placebo = proportion_confint(
    placebo_infarct_n, placebo_n, method="wilson"
)

print(
    "interval for aspirin infarct [%.4f, %.4f]"
    % (np.round(conf_interval_aspirin[0], 4), np.round(conf_interval_aspirin[1], 4))
)
print(
    "interval for placebo infarct [%.4f, %.4f]"
    % (np.round(conf_interval_placebo[0], 4), np.round(conf_interval_placebo[1], 4))
)

#%%
def proportions_confint_diff_ind(p1, count1, p2, count2, alpha=0.05):
    z = norm.ppf(1 - alpha / 2.0)

    left_boundary = (p1 - p2) - z * np.sqrt(
        p1 * (1.0 - p1) / count1 + p2 * (1 - p2) / count2
    )
    right_boundary = (p1 - p2) + z * np.sqrt(
        p1 * (1.0 - p1) / count1 + p2 * (1 - p2) / count2
    )

    return (left_boundary, right_boundary)


# Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина.
# Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.
#%%
conf_interval_diff = proportions_confint_diff_ind(
    placebo_p, placebo_n, aspirin_p, aspirin_n
)
print(
    "interval for aspirin and placebo probability diff: [%.4f, %.4f]"
    % (np.round(conf_interval_diff[0], 4), np.round(conf_interval_diff[1], 4))
)
# Продолжим анализировать данные эксперимента Гарвардской медицинской школы.
# Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину p/(1−p), которая называется шансами
# (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить p^. Например, шансы инфаркта в контрольной
# группе, принимавшей плацебо, можно оценить как
# (189/11034)/(1−189/11034)=189/(11034−189)≈0.0174
# Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх
# знаков после десятичной точки.
#%%
def calculate_odds(success_count, full_count):
    return float(success_count) / float(full_count - success_count)


#%%
aspirin_odds = calculate_odds(aspirin_infarct_n, aspirin_n)
placebo_odds = calculate_odds(placebo_infarct_n, placebo_n)
print("Placebo to aspirin odds ratio: %.4f" % np.round(placebo_odds / aspirin_odds, 4))

# Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения
# шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница?
# Округлите ответ до 4 знаков после десятичной точки.

# Чтобы получить в точности такой же доверительный интервал, как у нас:
# - составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы,
#   а потом все нули;
# - установите random seed=0;
# - сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.
#%%
aspirin_vector = np.append(
    np.ones(int(aspirin_infarct_n), dtype=int),
    np.zeros(int(aspirin_n - aspirin_infarct_n), dtype=int),
)
placebo_vector = np.append(
    np.ones(int(placebo_infarct_n), dtype=int),
    np.zeros(int(placebo_n - placebo_infarct_n), dtype=int),
)
#%%
def get_bootstrap_samples(data, n_samples):
    data_length = len(data)
    indices = np.random.randint(0, data_length, (n_samples, data_length))
    return data[indices]


#%%
np.random.seed(0)
aspirin_samples = get_bootstrap_samples(aspirin_vector, 1000)
placebo_samples = get_bootstrap_samples(placebo_vector, 1000)
#%%
def calculate_odds_from_samples(samples):
    samples_count = len(samples)
    success_count = len(samples[samples == 1])
    return calculate_odds(success_count, samples_count)


#%%
all_aspirin_odds = list(map(calculate_odds_from_samples, aspirin_samples))
all_placebo_odds = list(map(calculate_odds_from_samples, placebo_samples))

#%%
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2.0, 100 * (1 - alpha / 2.0)])
    return boundaries


#%%
odds = list(map(
    lambda x: x[1] / x[0],
    zip(all_aspirin_odds, all_placebo_odds),
))
#%%
print(np.round(stat_intervals(odds, 0.05), 4))

3 sigma rule refined: -2.9677
Aspirin infarct probability: 0.0094
Placebo infarct probability: 0.0171
Probability diff 0.0077
interval for aspirin infarct [0.0078, 0.0114]
interval for placebo infarct [0.0149, 0.0197]
interval for aspirin and placebo probability diff: [0.0047, 0.0107]
Placebo to aspirin odds ratio: 1.8321
[1.4442 2.3432]
